# Masked Language Model

Um Masked Language Model (MLM) é um tipo de modelo de linguagem amplamente utilizado em processamento de linguagem natural.

Durante o treinamento, uma parte dos tokens (palavras ou subpalavras) no texto de entrada é substituída por um token especial de máscara, como "[MASK]". O objetivo do modelo é prever corretamente quais eram os tokens originais que foram mascarados.

Essa estratégia obriga o modelo a aprender contextos ricos e relações entre as palavras, o que é fundamental para o desempenho em diversas tarefas, como análise de sentimentos, tradução, e resposta a perguntas. Modelos famosos que utilizam essa técnica incluem o BERT, que demonstrou ganhos significativos em várias benchmarks de NLP .


## IMBD

Para este projeto, será utilizado a base do IMBD.

## Configuração

In [6]:
import os

os.environ["KERAS_BACKEND"] = "torch"  # or jax, or tensorflow

import keras_hub

import keras
from keras import layers
from keras.layers import TextVectorization

from dataclasses import dataclass
import pandas as pd
import numpy as np
import glob
import re
from pprint import pprint

In [7]:
@dataclass
class Config:
    MAX_LEN = 256
    BATCH_SIZE = 32
    LR = 0.001
    VOCAB_SIZE = 30000
    EMBED_DIM = 128
    NUM_HEAD = 8  # used in bert model
    FF_DIM = 128  # used in bert model
    NUM_LAYERS = 1


config = Config()

# Carregando os dados

Primeiro, vamos carregar os dados que estão na pasta "aclImbd".

Duas funções serão utilizadas para isso:

- Uma irá criar uma lista contendo o conteúdo dos arquivos.
- A outra ficará responsável por criar um dataframe.

In [8]:
def get_text_list_from_files(files) -> list[str]:
    """
       Esta função irá retornar uma lista contendo todas as frases dos arquivos.
    """
    text_list: list[str] = []
    for name in files:
        with open(name) as f:
            for line in f:
                text_list.append(line)
    return text_list


def get_data_from_text_files(folder_name):

    # Arquivos de texto com avaliações positivas e negativas
    pos_files = glob.glob(f"aclImdb/{folder_name}/pos/*.txt")
    neg_files = glob.glob(f"aclImdb/{folder_name}/neg/*.txt")

    # Listas com as avaliações
    pos_texts: list[str] = get_text_list_from_files(pos_files)
    neg_texts: list[str] = get_text_list_from_files(neg_files)

    # Criação de um dataframe, com colunas chamadas "review" e "sentiment"
    df = pd.DataFrame(
        {
            "review": pos_texts + neg_texts,
            # todo Entender o que ele faz aqui!!
            "sentiment": [0] * len(pos_texts) + [1] * len(neg_texts),
        }
    )
    df = df.sample(len(df)).reset_index(drop=True)
    return df


train_df = get_data_from_text_files("train")
test_df = get_data_from_text_files("test")

all_data = pd.concat([train_df, test_df], ignore_index=True)

In [10]:
train_df.head()

,review,sentiment
0,On the face of it a film about women wanting t...,0
1,"Cinematography--Compared to 'The Wrestler,' a ...",0
2,I am very open to foreign films and like to th...,1
3,Name just says it all. I watched this movie wi...,0
4,This was what black society was like before th...,0
